In [133]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [134]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input: 28; Output: 28, RF: 3
        # self.conv2 = nn.Conv2d(32, 64, 3, padding=1) #input: 28; Output: 28, RF: 5
        # self.pool1 = nn.MaxPool2d(2, 2) #input: 28; Output: 14, RF: 6
        # # self.fc1 = nn.Linear(in_features = 1792 * 14, out_features = 64)
        # self.conv3 = nn.Conv2d(64, 128, 3, padding=1) #input: 14; Output: 14, RF: 10
        # self.conv4 = nn.Conv2d(128, 256, 3, padding=1) #input: 14; Output: 14, RF: 14
        # self.pool2 = nn.MaxPool2d(2, 2) #input: 14; Output: 7, RF: 16
        # self.conv5 = nn.Conv2d(256, 512, 3) #input: 7; Output: 5, RF: 24
        # self.conv6 = nn.Conv2d(512, 1024, 3) #input: 5; Output: 3, RF: 32
        # self.conv7 = nn.Conv2d(1024, 10, 3) #input: 3; Output: 1, RF: 40

        # # reducing channels to 8
        # self.conv1 = nn.Conv2d(1, 8, 3, padding=1) #input: 28; Output: 28, RF: 3
        # self.bn1 = nn.BatchNorm2d (8)
        # self.conv2 = nn.Conv2d(8, 16, 3, padding=1) #input: 28; Output: 28, RF: 5
        # self.bn2 = nn.BatchNorm2d (16)
        # self.pool1 = nn.MaxPool2d(2, 2) #input: 28; Output: 14, RF: 6
        # self.antman1 = nn.Conv2d(16, 16, 1)
        # self.conv3 = nn.Conv2d(16, 32, 3, padding=1) #input: 14; Output: 14, RF: 10
        # self.bn3 = nn.BatchNorm2d (32)
        # self.conv4 = nn.Conv2d(32, 64, 3, padding=1) #input: 14; Output: 14, RF: 14
        # self.bn4 = nn.BatchNorm2d (64)
        # self.pool2 = nn.MaxPool2d(2, 2) #input: 14; Output: 7, RF: 16
        # self.antman2 = nn.Conv2d(64, 64, 1)
        # self.conv5 = nn.Conv2d(64, 128, 3) #input: 7; Output: 5, RF: 24
        # self.bn5 = nn.BatchNorm2d (128)
        # self.conv6 = nn.Conv2d(128, 256, 3) #input: 5; Output: 3, RF: 32
        # self.bn6 = nn.BatchNorm2d (256)
        # self.conv7 = nn.Conv2d(256, 10, 3) #input: 3; Output: 1, RF: 40
        # #self.dropout = nn.Dropout(0.1)

        # changing antman to reduce output
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1) #input: 28; Output: 28, RF: 3
        self.bn1 = nn.BatchNorm2d (8)
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1) #input: 28; Output: 28, RF: 5
        self.bn2 = nn.BatchNorm2d (16)
        self.pool1 = nn.MaxPool2d(2, 2) #input: 28; Output: 14, RF: 6
        self.antman1 = nn.Conv2d(16, 8, 1)
        self.conv3 = nn.Conv2d(8, 16, 3, padding=1) #input: 14; Output: 14, RF: 10
        self.bn3 = nn.BatchNorm2d (16)
        self.conv4 = nn.Conv2d(16, 32, 3, padding=1) #input: 14; Output: 14, RF: 14
        self.bn4 = nn.BatchNorm2d (32)
        self.pool2 = nn.MaxPool2d(2, 2) #input: 14; Output: 7, RF: 16
        self.antman2 = nn.Conv2d(32, 8, 1)
        self.conv5 = nn.Conv2d(8, 16, 3) #input: 7; Output: 5, RF: 24
        self.bn5 = nn.BatchNorm2d (16)
        self.conv6 = nn.Conv2d(16, 32, 3) #input: 5; Output: 3, RF: 32
        self.bn6 = nn.BatchNorm2d (32)
        self.conv7 = nn.Conv2d(32, 10, 3) #input: 3; Output: 1, RF: 40
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        # x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.conv1(x)
        x = F.relu(x)
        x = self.bn1(x)
        x = self.dropout(x)

        x = self.conv2(x)
        x = F.relu(x)
        x = self.bn2(x)
        x = self.dropout(x)

        x = self.pool1(x)
        x = self.antman1(x)

        # x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = self.conv3(x)
        x = F.relu(x)
        x = self.bn3(x)
        x = self.dropout(x)

        x = self.conv4(x)
        x = F.relu(x)
        x = self.bn4(x)
        x = self.dropout(x)

        x = self.pool2(x)
        x = self.antman2(x)

        # x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = self.conv5(x)
        x = F.relu(x)
        x = self.bn5(x)
        x = self.dropout(x)

        x = self.conv6(x)
        x = F.relu(x)
        x = self.bn6(x)
        x = self.dropout(x)

        # x = F.relu(self.conv7(x))
        x = self.conv7(x)

        x = x.view(-1, 10)
        return F.log_softmax(x)

In [135]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
           Dropout-3            [-1, 8, 28, 28]               0
            Conv2d-4           [-1, 16, 28, 28]           1,168
       BatchNorm2d-5           [-1, 16, 28, 28]              32
           Dropout-6           [-1, 16, 28, 28]               0
         MaxPool2d-7           [-1, 16, 14, 14]               0
            Conv2d-8            [-1, 8, 14, 14]             136
            Conv2d-9           [-1, 16, 14, 14]           1,168
      BatchNorm2d-10           [-1, 16, 14, 14]              32
          Dropout-11           [-1, 16, 14, 14]               0
           Conv2d-12           [-1, 32, 14, 14]           4,640
      BatchNorm2d-13           [-1, 32, 14, 14]              64
          Dropout-14           [-1, 32,

<ipython-input-134-cc08b614d854>:99: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [136]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [137]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [138]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-134-cc08b614d854>:99: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.07402382791042328 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 27.92it/s]



Test set: Average loss: 0.1028, Accuracy: 9697/10000 (97%)



loss=0.005928864702582359 batch_id=468: 100%|██████████| 469/469 [00:17<00:00, 26.78it/s]



Test set: Average loss: 0.0470, Accuracy: 9857/10000 (99%)



loss=0.010493314824998379 batch_id=468: 100%|██████████| 469/469 [00:17<00:00, 26.92it/s]


...


Test set: Average loss: 0.0217, Accuracy: 9926/10000 (99%)



loss=0.005983587354421616 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 27.92it/s]



Test set: Average loss: 0.0204, Accuracy: 9931/10000 (99%)



loss=0.049422238022089005 batch_id=468: 100%|██████████| 469/469 [00:17<00:00, 26.93it/s]



Test set: Average loss: 0.0218, Accuracy: 9923/10000 (99%)

